In [1]:
pip install datasets sentence_transformers cohere numpy pinecone tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.8/173.8 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [2]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer,util
import cohere
import numpy as np
import pinecone
from tqdm import tqdm
import os
import warnings
from IPython.display import display
warnings.filterwarnings("ignore")

### API KEYS


In [3]:
COHERE_API_KEY = "INdVNJcXJYO5nNQHovgyn966o25uXp7ncltupjhQ"
PINECONE_API_KEY = "46c01a13-e885-432d-9c5f-5a96248cbff2"

## **3.1.1 - Report**

### **Dataset Selection:**

The **SQuAD (Stanford Question Answering Dataset)** is a popular dataset in the field of natural language processing (NLP) and machine reading comprehension.

It consists of a large collection of question-answer pairs, where each question is posed on a specific context paragraph from a set of Wikipedia articles.



**Here’s an overview and rationale for using the SQuAD dataset for your task:**
-   Each instance in SQuAD consists of a context paragraph from Wikipedia and a set of questions with corresponding answers that are spans of text within the context paragraph.
-   The dataset is split into train and dev sets, where the train set is used for training and the dev set for evaluating models.
Relevance to Task:
-   Since your task involves building a Retrieval-Augmented Generation (RAG) pipeline that retrieves relevant documents based on queries and generates answers, SQuAD provides a suitable corpus. It contains well-annotated, factual information that can serve as contexts for generating answers.
-   With its diverse range of topics and questions, including both simple and nuanced inquiries, SQuAD challenges your RAG pipeline to handle various types of queries and contexts effectively.


In summary, the SQuAD dataset is chosen for its relevance to question answering tasks, its availability for research purposes, and its ability to provide a robust evaluation framework for assessing the effectiveness of your Retrieval-Augmented Generation pipeline.



### Prepering the **Embedding Model**, **VectorDB** & **LLM**

In [4]:
# Constants
DATASET_NAME = 'squad'
COHERE_API_KEY = "INdVNJcXJYO5nNQHovgyn966o25uXp7ncltupjhQ"
PINECONE_API_KEY = "46c01a13-e885-432d-9c5f-5a96248cbff2"
EMBEDDING_MODEL = 'all-MiniLM-L6-v2'
INDEX_NAME = "squad-index"

In [5]:

# Initialize SentenceTransformer with GPU support
model = SentenceTransformer(EMBEDDING_MODEL)
co = cohere.Client(api_key=COHERE_API_KEY)

# Function to preprocess documents
def preprocess_document(text):
    """Preprocesses a document text by stripping and lowercasing."""
    return text.strip().lower()

# Function to create Pinecone index
def create_pinecone_index(api_key, index_name, dimension, metric='cosine'):
    """
    Creates or retrieves a Pinecone index.
    Args:
        api_key: Pinecone API key.
        index_name: Name of the index to create or retrieve.
        dimension: Dimensionality of the embeddings.
        metric: Similarity metric for the index.
    Returns:
        Pinecone index object.
    """
    pc = pinecone.Pinecone(api_key=api_key)
    if index_name not in pc.list_indexes().names():
        pc.create_index(
            name=index_name,
            dimension=dimension,
            metric=metric,
            spec=pinecone.ServerlessSpec(
                cloud='aws',
                region='us-east-1'
            )
        )
    return pc.Index(index_name)

# Function to upsert vectors into Pinecone index
def upsert_vectors(index, embeddings, dataset, text_field='context', batch_size=128):
    """
    Upserts embeddings and corresponding metadata into a Pinecone index.
    Args:
        index: Pinecone index object.
        embeddings: NumPy array of embeddings.
        dataset: Dataset containing the metadata.
        text_field: Field name containing the text data.
        batch_size: Batch size for upsert operations.
    Returns:
        Updated Pinecone index object.
    """
    print("Upserting the embeddings to the Pinecone index...")
    shape = embeddings.shape

    ids = [str(i) for i in range(shape[0])]
    meta = [{text_field: preprocess_document(text)} for text in dataset[text_field]]

    to_upsert = list(zip(ids, embeddings, meta))

    for i in tqdm(range(0, shape[0], batch_size)):
        i_end = min(i + batch_size, shape[0])
        index.upsert(vectors=to_upsert[i:i_end])
    return index

# Function to augment prompt with retrieved documents
def augment_prompt(query, model, index, top_k=3):
    """
    Augments the query prompt with retrieved documents from the Pinecone index.
    Args:
        query: Query string.
        model: SentenceTransformer model for encoding queries.
        index: Pinecone index object.
        top_k: Number of top documents to retrieve.
    Returns:
        Augmented prompt and retrieved source knowledge as strings.
    """
    query_embedding = model.encode(query).tolist()
    results = index.query(vector=query_embedding, top_k=top_k, include_metadata=True)['matches']
    text_matches = [match['metadata']['context'] for match in results]
    source_knowledge = "\n\n".join(text_matches)
    augmented_prompt = f"""Using the contexts below, answer the query.
    Contexts:
    {source_knowledge}
    If the answer is not included in the source knowledge - say that you don't know.
    Query: {query}"""
    return augmented_prompt, source_knowledge

# Function to load and embed dataset for RAG pipeline
def load_and_embed_dataset(dataset_name='squad', split='train', model_name='all-MiniLM-L6-v2', text_field='context', rec_num=87600):
    """
    Loads and embeds a dataset using SentenceTransformer.
    Args:
        dataset_name: Name of the dataset.
        split: Split of the dataset to load.
        model_name: Name of the SentenceTransformer model to use.
        text_field: Field name containing the text data.
        rec_num: Number of records to process.
    Returns:
        Loaded dataset and corresponding embeddings.
    """
    print("Loading and embedding the dataset...")

    # Load the dataset
    dataset = load_dataset(dataset_name, split=split)

    # Embed the first `rec_num` rows of the dataset
    embeddings = model.encode(dataset[text_field][:rec_num])

    print("Done!")
    return dataset, embeddings

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### **Example case of QA model failed & the RAG pipline succeeded:**

In [6]:
### Standard QA Model ###
def standard_qa_model(query, questions, answers):
    """
    Performs standard question answering using a list of questions and answers.
    Args:
        query: Query string.
        questions: List of questions from the dataset.
        answers: List of corresponding answers from the dataset.
    Returns:
        Answer string if found, otherwise "Answer not found.".
    """
    for idx, q in enumerate(questions):
        if query.lower() in q.lower():
            return answers[idx]['text']
    return "Answer not found."

# Load SQuAD dataset
dataset = load_dataset(DATASET_NAME, split='train')
questions = dataset['question']
answers = dataset['answers']

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [14]:
### RAG Pipeline ###
def run_rag_pipeline(query, model, co, dataset_name='squad', split='train', model_name='all-MiniLM-L6-v2', text_field='context', rec_num=4000):
    """
    Runs the Retrieval-Augmented Generation (RAG) pipeline for question answering.
    Args:
        query: Query string.
        model: SentenceTransformer model for encoding queries.
        co: Cohere client for generating answers.
        dataset_name: Name of the dataset.
        split: Split of the dataset to load.
        model_name: Name of the SentenceTransformer model to use.
        text_field: Field name containing the text data.
        rec_num: Number of records to process.
    Returns:
        RAG pipeline response as a string.
    """
    # Load and embed dataset
    dataset, document_embeddings = load_and_embed_dataset(dataset_name, split, model_name, text_field, rec_num)

    # Create or retrieve Pinecone index
    index = create_pinecone_index(PINECONE_API_KEY, INDEX_NAME, document_embeddings.shape[1])

    # Upsert embeddings into Pinecone index
    index = upsert_vectors(index, document_embeddings, dataset)

    # Augment prompt with retrieved documents
    augmented_prompt, _ = augment_prompt(query, model, index)

    # Generate answer using RAG pipeline
    rag_response = co.chat(model='command-r-plus', message=augmented_prompt).text

    return rag_response

In [20]:
# Example query
query_list = [
    "Who was the first woman to win a Nobel Prize?",
    "How has the COVID-19 pandemic impacted global economic trends?",
    "What year did Albert Einstein publish his theory of general relativity?"
]

for query in query_list:
    print(f'{query}\n')


    # Run Standard QA Model
    standard_response = standard_qa_model(query, questions, answers)
    print(f"Standard QA Model Response: {standard_response}\n")

    # Run RAG pipeline
    rag_response = run_rag_pipeline(query, model, co)
    print(f"RAG Pipeline Response: {rag_response}\n")
    print("===============================================================\n")


Who was the first woman to win a Nobel Prize?

Standard QA Model Response: Answer not found.

Loading and embedding the dataset...


KeyboardInterrupt: 

### **Explation, insight & observations:**

**Retrieval System in RAG Pipeline**

In the Retrieval-Augmented Generation (RAG) pipeline, the retrieval system plays a critical role in fetching relevant information from a pre-indexed database based on the query provided.

Here’s how it works step-by-step:

-  **Vector Embeddings:** Documents (contexts) are first converted into high-dimensional vector representations using models like SentenceTransformer. These vectors encode the meaning of the text in a way that computers can understand.

-  **Pinecone Index:** These vector embeddings are then stored and managed using Pinecone, which is a specialized database designed for storing and searching vectors efficiently. Pinecone allows us to quickly find documents that are most similar to the query based on their semantic content.


**Prompts Used in RAG Pipeline**

The prompts used in the RAG pipeline are designed to enrich the query with retrieved documents, aiming to provide a broader context for generating accurate answers. Here are the key aspects of how prompts are structured:

-  **Augmentation:** Retrieved documents are integrated into the prompt to provide additional context. This integration helps the model understand the query better by including relevant information extracted from the indexed documents.

-  **Structured Format:** The prompts typically present the original query followed by the retrieved contexts. This format guides the model to generate answers that are informed by both the original question and the additional knowledge retrieved from the database.

**Elements Used in the Implementation**

**Modal**
-  **SentenceTransformer:** Utilized for encoding text into numerical vector representations suitable for semantic understanding.

**Metrics**
-  **Cosine Similarity:** Employed as a metric for measuring similarity between query embeddings and document embeddings within Pinecone.

**APIs and Libraries**
- **cohere.Client:** Integrated to generate answers using the Cohere API, specifically with the 'command-r-plus' model, leveraging retrieved knowledge.

- **pinecone.Pinecone:** Utilized for creating and managing the vector index, enabling efficient storage and retrieval operations.

**Constants and Configuration**
- **DATASET_NAME:** Refers to the name of the dataset ('squad' in this case) used for question answering tasks.

- **COHERE_API_KEY:** API key used for accessing the Cohere service for answer generation.

- **PINECONE_API_KEY:** API key used for interacting with Pinecone's vector database for document indexing.

- **EMBEDDING_MODEL:** Specifies the SentenceTransformer model ('all-MiniLM-L6-v2') employed for text embeddings.

- **INDEX_NAME:** Name assigned to the Pinecone index ('squad-index') where embeddings are stored and queried.


**Effectiveness Insights:**

- **Reduction of Hallucinations**

One significant benefit of the RAG pipeline is its ability to reduce "hallucinations"—situations where the model generates plausible but incorrect answers. By enriching the query with real-world data from retrieved documents, the pipeline helps the model generate responses that are grounded in factual information. This reduces the risk of speculative answers.

- **Improvement in Answer Accuracy**

Another advantage is the enhancement in answer accuracy. By incorporating retrieved documents into the prompt, the model gains access to a wider range of relevant information. This broader context allows the model to provide more precise and comprehensive answers, covering nuanced aspects of the query that might be missed in simpler question-answering approaches.

- **Conclusion**

While these observations provide valuable insights into the operational benefits of the RAG pipeline, a thorough evaluation across diverse datasets and scenarios would be necessary for conclusive findings. Nevertheless, leveraging retrieval mechanisms and augmented prompts holds promise in improving the reliability and accuracy of question answering systems by grounding answers in real-world contexts.

**Git link :** https://github.com/machpudmor/Part-3-.git